In [6]:
import cv2
import numpy as np

In [35]:
video_path = 'CCTV_TopDown.mp4'
cap = cv2.VideoCapture(video_path)

In [36]:
if cap.isOpened():
    print("yes, no error")


In [37]:
def preprocess_frame(frame):
    # Resize frame for faster processing
    frame_resized = cv2.resize(frame, (640, 480))
    return frame_resized

ret, frame = cap.read()
if ret:
    preprocessed_frame = preprocess_frame(frame)

### Import Yolov5

In [11]:
import torch

# Load the YOLOv5 model from the Ultralytics repository
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Optionally set a higher confidence threshold
model.conf = 0.5

Using cache found in C:\Users\Acer/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-8-29 Python-3.8.8 torch-2.3.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [38]:
def detect_vehicles(frame):
    # Perform vehicle detection
    results = model(frame)
    
    # Filter results to keep only vehicles (car, truck, bus, etc.)
    vehicle_classes = [2, 3, 5, 7]
    
    # Get the detections in [x1, y1, x2, y2, confidence, class] format
    detections = results.xyxy[0]
    
    # Filter out non-vehicle detections
    mask = detections[:, -1].cpu().numpy().astype(int)
    filtered_mask = [i for i, cls in enumerate(mask) if cls in vehicle_classes]
    
    # Apply the filter to the results object
    results.xyxy[0] = detections[filtered_mask]
    
    return results

In [25]:
results = detect_vehicles(preprocessed_frame)

In [16]:
!git clone https://github.com/abewley/sort.git

fatal: destination path 'sort' already exists and is not an empty directory.


In [14]:
cd sort

C:\Users\Acer\Desktop\Sunway\Task1\sort


In [15]:
import sys
sys.path.append('sort')

In [16]:
from sort import Sort

In [26]:
# Initialize SORT tracker
tracker = Sort()

In [27]:
def track_vehicles(results):
    detections = []
    for result in results:
        x1, y1, x2, y2, conf, cls = result
        detections.append([x1, y1, x2, y2, conf])

    # Update tracker with current frame detections
    tracks = tracker.update(np.array(detections))
    return tracks

#tracks = track_vehicles(results.xyxy[0].numpy())

In [39]:
def count_vehicles(tracks, line_position=300):
    vehicle_count = 0
    for track in tracks:
        x1, y1, x2, y2, track_id = track
        if y1 < line_position < y2:
            vehicle_count += 1
            # Mark this vehicle as counted
            # (implement additional logic if needed)
    return vehicle_count

#vehicle_count = count_vehicles(tracks)

In [40]:
def display_results(frame, tracks, vehicle_count):
    for track in tracks:
        x1, y1, x2, y2, track_id = track
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {int(track_id)}', (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    
    # Display the vehicle count
    cv2.putText(frame, f'Vehicle Count: {vehicle_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show the frame
    cv2.imshow('Frame', frame)


In [42]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess, detect, track, and count
    preprocessed_frame = preprocess_frame(frame)
    results = detect_vehicles(preprocessed_frame)
    tracks = track_vehicles(results.xyxy[0].numpy())
    vehicle_count = count_vehicles(tracks)

    # Display results
    display_results(preprocessed_frame, tracks, vehicle_count)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()